In [3]:
import os
import csv
import gzip
os.chdir("/Users/clarkbernier/Box Sync/IBM Local/ibm-topic-model/library/liwc_js")
%run jensen_shannon.py
docs = []
csv.field_size_limit(93000000)
#with open("../../outputs/exec_set_docs.csv", 'r') as csvfile: 
with gzip.open("../../outputs/exec_comps_set_docs.gz", 'r') as csvfile: 
    csv_file = csv.DictReader(csvfile)
    for row in csv_file:
        docs.append(row)
len(docs)

108

In [ ]:


jensen_shannon_distances(documents, doc_segmentation_fnc=lambda x: [x['frm']], 
                                pre_processed=False,
                                liwc_map=False, vocabsize=1000,
                                sampling=False, sampsize=1000,
                                min_segment_size=100,
                                verbose=False)

In [ ]:
targets = ["quote_set", "responded_set", "early_set", "exec_set", "mgr_set", "asia_set", "us_set", "euro_set"]
just_bigs = {}

for idx, msg in enumerate(docs):
    key = msg['user']
    if key in targets:
        just_bigs[key] = msg

dists = {}
for key in just_bigs.keys():
    dists[key] = get_term_count_distribution([just_bigs[key]], vocabsize=vocabsize, liwc_map=liwc_map, pre_processed=pre_processed)


In [4]:
vocabsize=4000
sampling=False 
sampsize=1000
verbose=False
pre_processed=False
liwc_map=True

segments2docs = {}
doc_segmentation_fnc=lambda x: [x['user']]
documents = docs
keys = {}

for idx, msg in enumerate(documents):
    if verbose:
        sys.stderr.write('\r') ; sys.stderr.write('msg %s' % idx) ; sys.stderr.flush()
    key = doc_segmentation_fnc(msg)[0]
    segments2docs[key] = msg
if verbose: sys.stderr.write('\n')

# For each segment, get count distribution over terms:
dists = {}
for key in segments2docs.keys():
    print key
    dists[key] = get_term_count_distribution([segments2docs[key]], vocabsize=vocabsize, liwc_map=liwc_map, pre_processed=pre_processed)


#return distances

# Now measure pairwise distances:
# dist_from_exec = {}
# for key in dists.keys():
#    dist_from_exec[key] = jensen_shannon(dists[key], dists["quote_set"])


In [12]:
# Caluculate an Overall Center
seg2docs_copy = segments2docs.copy()
dud = seg2docs_copy.pop("exec_set")
mega_comment = {'text':" ".join([d['text'] for d in seg2docs_copy.values()])}
overall_center = get_term_count_distribution([mega_comment], vocabsize=vocabsize, liwc_map=liwc_map, pre_processed=pre_processed)

In [33]:
print(jensen_shannon(dists["quote_set"], overall_center))
print(jensen_shannon(overall_center, dists["exec_set"]))
print(jensen_shannon(overall_center, dists["mgr_set"]))
print(jensen_shannon(dists["quote_set"], dists["exec_set"]))
print(jensen_shannon(dists["quote_set"], dists["mgr_set"]))
print(jensen_shannon(dists["exec_set"], dists["mgr_set"]))
print(jensen_shannon(overall_center, dists["asia_set"]))
print(jensen_shannon(overall_center, dists["euro_set"]))
print(jensen_shannon(overall_center, dists["us_set"]))
print(jensen_shannon(dists["quote_set"], dists["asia_set"]))
print(jensen_shannon(dists["quote_set"], dists["euro_set"]))
print(jensen_shannon(dists["quote_set"], dists["us_set"]))
print(jensen_shannon(dists["quote_set"], dists["early_set"]))

0.032575943585981995
0.03180867029412797
0.009102367940830431
0.04024722742953861
0.02946740997116304
0.0279816833870955
0.02188255978891531
0.010813704192912322
0.007160448749681352
0.035231822515628286
0.03262139400059169
0.03479237685507478
0.11282687019729255


In [29]:
# Persist the Overall Center and the other Target Centers

targets = ["quote_set", "responded_set", "early_set", "exec_set", "mgr_set", "asia_set", "us_set", "euro_set"]

out_set = {}
for key in overall_center:
    ret_set = [overall_center[key]]
    for target_key in targets:
        ret_set.append(dists[target_key].get(key) or 0)
    out_set[key] = ret_set
with open('../../outputs/liwc_sets/liwc_distributions.csv','wb') as f:    
    w = csv.writer(f)
    w.writerow(["liwc.cat","overall.dist"] + targets)
    for k,v in out_set.items():
       w.writerow([k] + list(v))

In [40]:
# Now measure and output the pairwise distances
dists["overall_center"] = overall_center
targets = ["quote_set", "responded_set", "early_set", "exec_set", "mgr_set", "asia_set", "us_set", "euro_set", "overall_center", "comp87", "comp22"]

dist_set = {}
for target_key in targets:
    print target_key
    for key in dists.keys():
        dist_set[key] = jensen_shannon(dists[key], dists[target_key])
    fil_nam = '../../outputs/liwc_sets/dists_from_%s.csv' % (target_key) 
    with open(fil_nam,'wb') as f:    
        w = csv.writer(f)
        headr = "dist.from.%s" % (target_key)
        w.writerow(["user","dist"])
        w.writerows(dist_set.items())

quote_set
responded_set
early_set
exec_set
mgr_set
asia_set
us_set
euro_set
overall_center
comp87
comp22


In [39]:
target_key = targets[2]
key = dists.keys()[1]
print key
print target_key
dist_set = {}
dist_set[key] = jensen_shannon(dists[key], dists[target_key])

print dist_set


quote_set
early_set
{'quote_set': 0.11282687019729255}


In [ ]:
# what do the execs look like?
print(dists["exec_set"])

In [ ]:
LIWC

In [ ]:
LIWC.items()

In [ ]:
len(just_bigs)

In [ ]:
just_bigs.keys()


In [ ]:
len(overall_center)

In [ ]:
len(docs)

In [ ]:
dist_from_quote